In [1]:
import os
import cv2
from tqdm import tqdm
from keras.models import load_model
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

DATA_DIR = './data/test/'
CATEGORIES = ['Shoplifting', 'Normal_Activity']
outputmodelpath = r'model/VGG16/'

In [2]:
def preprocess_data_to_images(IMG_SIZE = 224):
    count = 0
    counter = 1
    frames = []

    for c in CATEGORIES:
        path = os.path.join(DATA_DIR, c)
        class_num = CATEGORIES.index(c)

        for vid in tqdm(os.listdir(path)):
            try:
                vid = os.path.join(path, vid)
                cap = cv2.VideoCapture(vid)
                count = 0
                counter += 1
                success = True
                while success:
                    success,image = cap.read()
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    #                 print('read a new frame:',success)
                    if count%30 == 0 :
                        frames.append([image, class_num])
                        cv2.imwrite(DATA_DIR + c +'/frame%d.jpg'%count,image)
                    count+=1
            except Exception as e:
                pass

    print(count, " frames extracted")
    # frames = np.array(frames)
    # print("data shape =\t", frames.shape)

In [3]:
def preprocess_data(IMG_SIZE = 224, data=[], labels=[]):
    for c in CATEGORIES:
        path = os.path.join(DATA_DIR, c)
        class_num = CATEGORIES.index(c)

        for img in  tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                img_resized = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([img_resized, class_num])
                labels.append(CATEGORIES[class_num])
            except Exception as e:
                pass
    print("--------------------------")
    print("Images loaded successfully!")
    print(f"Found {len(data)} images and {len(labels)} labels from {len(CATEGORIES)} Categories")
    #data = np.array(data)
    #np.random.shuffle(data)
    return data, labels

In [4]:
data, labels = preprocess_data()

100%|█████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 728.07it/s]

--------------------------
Images loaded successfully!
Found 196 images and 196 labels from 2 Categories


In [5]:
X = [datum[0] for datum in data]
y = [datum[1] for datum in data]

X = np.array(X)
y = np.array(y)

In [ ]:
model = load_model(outputmodelpath)

In [ ]:
score = model.evaluate(X, y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y, y_pred)
print(cm)

In [ ]:
sns.heatmap(cm, annot=True, cmap='Blues')